In [2]:
from datasets import load_dataset, concatenate_datasets
yandex = load_dataset("its5Q/yandex-q", split="train")
diasum = load_dataset("bragovo/diasum")
# diasum - concatenation of train, test splits
diasum = concatenate_datasets([diasum["train"], diasum["test"]]) 

Found cached dataset yandex-q (/home/alexw/.cache/huggingface/datasets/its5Q___yandex-q/default/0.1.0/42efa0b5c9064323024876f0df999e68eb9bebe46fc4fb946594004e53efcb0a)
Using custom data configuration bragovo--diasum-0f75c6ea734d87f9


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/alexw/.cache/huggingface/datasets/bragovo___json/bragovo--diasum-0f75c6ea734d87f9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset_builder, get_dataset_split_names
from pprint import pprint

datasets_names = ["its5Q/yandex-q", 'bragovo/diasum']
for dataset_name in datasets_names:
    build = load_dataset_builder(dataset_name)
    # print split names, size (amount of samples) and features
    print(f"Dataset: {dataset_name}")
    pprint(build.info.splits)
    print(f"Features: {build.info.features}")
    print(f"Size: {build.info.splits.total_num_examples}")

In [4]:
pprint(yandex[0])

{'answer': 'Никак но можно попробовать найти сайты которые за тебя пройдут '
           'капчу за тебя попробуй это а так не вариант ну или зайди с другого '
           'браузера если с пк и пробуй опять',
 'description': '',
 'question': 'Как войти в Роблокс, если не работает верификация, пишет слишком '
             'медленно?'}


In [5]:
pprint(diasum[0])

{'dialogue': 'Брэд: Эй, ты помнишь меня?\r\n'
             'Клаудия: Нет, не совсем.\r\n'
             'Клаудия: Я должна помнить?\r\n'
             'Брэд: Мы встретились на конференции в прошлом месяце.\r\n'
             'Брэд: Ты должна помнить меня.\r\n'
             'Клаудия: Подожди секунду...\r\n'
             'Клаудия: О, да, Брэд из утренней сессии.\r\n'
             'Клаудия: Должно быть, у тебя есть причина, почему ты пишешь.\r\n'
             'Брэд: Да, я понравился твоя презентация, но я забыл спросить '
             'тебя о источниках, которые ты использовал.\r\n'
             'Клаудия: Спасибо за любезные слова. Вот:\r\n'
             'Клаудия: <file_other>\r\n'
             'Брэд: Большое спасибо. Это очень много значит.\r\n'
             'Клаудия: Рада, что смогла помочь :)\r\n'
             'Брэд: До свидания :)',
 'id': '15037',
 'summary': 'Брэд и Клаудия встретились на конференции. Брэд понравилась '
            'презентация Клаудии и он хочет знать источники, котор

In [6]:
import numpy as np
# Looking on dialogues you can note that each replica is separated by '\n' and each speaker is separated by ': '
n = 1
for i, dialog in enumerate(np.random.choice(diasum, n)):
    print(dialog['dialogue'])

Джошуа: посмотри в окно
Ноа: что там? XD
Джошуа: сделай это :P
Ноа: о милости моей!
Ноа: так много снега :D Когда это произошло лол
Джошуа: :D
Ноа: пора строить снеговика!
Джошуа: построй Олафа :d
Ноа: конечно, я сделаю, он мой идол!
Джошуа: XD


In [7]:
def format_dialogue(history, speech1, speech2,  user1=1, user2=2):
    return f'history: {history}\nspeaker{user1}: {speech1}\tspeaker{user2}: {speech2}'

def proceed_dialogue(dialogue, sep=': '):
    # remove empty lines
    lines = list(filter(lambda x: x.strip(), dialogue.split('\n'))) 
    # split by first ': ' to get speaker and text. Lower and strip text
    splits = list(map(lambda x: x.lower().strip().split(sep, 1), lines)) 
    splits = list(filter(lambda x: len(x) == 2, splits)) # remove lines without ': '
    res = [] # list of dialogues
    history = '' # history of dialogues
    prev_speaker, prev_text = splits[0] # first speaker and text

    dialogue_dict = {prev_speaker: 1} # {speaker: number}
    last = 2 # next number
    for speaker, text in splits[1:]:
        speaker, text = speaker.strip(), text.strip()
        if speaker not in dialogue_dict:
            dialogue_dict[speaker] = last
            last += 1
        res.append(format_dialogue(history, prev_text, text, dialogue_dict[prev_speaker], dialogue_dict[speaker]))
        history += f'speaker{dialogue_dict[prev_speaker]}: {prev_text}\t'
        prev_speaker, prev_text = speaker, text
    return res

n = np.random.randint(0, len(diasum))
dialogue = diasum[n]['dialogue']
print(f'Before\n{dialogue}')
res = proceed_dialogue(dialogue)

print(f'After')
for i, line in enumerate(res):
    print(f'--------{i}--------')
    print(line)



Before
Али: парень, мне нужен этот жесткий диск
Кейн: приходи за ним
Али: Хорошо, буду там через минуту
Кейн: конечно..
After
--------0--------
history: 
speaker1: парень, мне нужен этот жесткий диск	speaker2: приходи за ним
--------1--------
history: speaker1: парень, мне нужен этот жесткий диск	
speaker2: приходи за ним	speaker1: хорошо, буду там через минуту
--------2--------
history: speaker1: парень, мне нужен этот жесткий диск	speaker2: приходи за ним	
speaker1: хорошо, буду там через минуту	speaker2: конечно..


In [8]:
def yandex_preprocess(elem):
    # Here will be no history
    elem['question'], elem['answer'] = elem['question'].lower().strip(), elem['answer'].lower().strip()
    return {'texts': [format_dialogue('', elem['question'], elem['answer'])]}

# Throws error: proceed_dialogue isn't defined. That's because I use several processes to preprocess data
def diasum_preprocess(elem):
    dialogue = elem['dialogue']
    texts = proceed_dialogue(dialogue)
    return {'texts': texts}

# def diasum_preprocess(elem, sep=': '):
#     dialogue = elem['dialogue']
#     # remove empty lines
#     lines = list(filter(lambda x: x.strip(), dialogue.split('\n'))) 
#     # split by first ': ' to get speaker and text. Lower and strip text
#     splits = list(map(lambda x: x.lower().strip().split(sep, 1), lines)) 

#     res = [] # list of dialogues
#     history = '' # history of dialogues
#     prev_speaker, prev_text = splits[0] # first speaker and text

#     dialogue_dict = {prev_speaker: 1} # {speaker: number}
#     last = 2 # next number
#     for speaker, text in splits[1:]:
#         speaker, text = speaker.strip(), text.strip()
#         if speaker not in dialogue_dict:
#             dialogue_dict[speaker] = last
#             last += 1
#         res.append(format_dialogue(history, prev_text, text, dialogue_dict[prev_speaker], dialogue_dict[speaker]))
#         history += f'{dialogue_dict[prev_speaker]}: {prev_text}\t'
#         prev_speaker, prev_text = speaker, text
#     return res


In [9]:
n = np.random.randint(0, len(diasum))
pprint(diasum_preprocess(diasum[n]))

{'texts': ['history: \n'
           'speaker1: эй, джейк, есть новости про выходные?\tspeaker2: сьюзан '
           'приедет в пятницу утром. нам нужно пойти к ее матери.',
           'history: speaker1: эй, джейк, есть новости про выходные?\t\n'
           'speaker2: сьюзан приедет в пятницу утром. нам нужно пойти к ее '
           'матери.\tspeaker1: ок. а где она сейчас?',
           'history: speaker1: эй, джейк, есть новости про выходные?\t'
           'speaker2: сьюзан приедет в пятницу утром. нам нужно пойти к ее '
           'матери.\t\n'
           'speaker1: ок. а где она сейчас?\tspeaker2: дома. она в порядке, но '
           'немного скучает... каждый день к ней приходит медсестра, но она не '
           'может выйти на улицу.',
           'history: speaker1: эй, джейк, есть новости про выходные?\t'
           'speaker2: сьюзан приедет в пятницу утром. нам нужно пойти к ее '
           'матери.\tspeaker1: ок. а где она сейчас?\t\n'
           'speaker2: дома. она в порядке,

In [9]:
pprint(yandex_preprocess(yandex[0]))

{'texts': ['history: \n'
           'speaker1: как войти в роблокс, если не работает верификация, пишет '
           'слишком медленно?\tspeaker2: никак но можно попробовать найти '
           'сайты которые за тебя пройдут капчу за тебя попробуй это а так не '
           'вариант ну или зайди с другого браузера если с пк и пробуй опять']}


In [10]:
# We do need batches for speedup but only when tokenizing
diasum_texts = diasum.map(diasum_preprocess, batched=False, remove_columns=diasum.column_names)
yandex_texts = yandex.map(yandex_preprocess, batched=False, remove_columns=yandex.column_names)

Loading cached processed dataset at C:\Users\rodio\.cache\huggingface\datasets\its5Q___yandex-q\default\0.1.0\42efa0b5c9064323024876f0df999e68eb9bebe46fc4fb946594004e53efcb0a\cache-d1a9b403a888ccf3.arrow


In [11]:
yandex_texts[4]

{'texts': ['history: \nspeaker1: почему у ноутбука очень часто включается вентилятор?\tspeaker2: если у вас открыто 20 вкладок это особо не нагружает ноутбук, просто забивается оперативная память. открытое видео нагружает видео процессор. часто включается вентилятор понятие очень растяжимое тк зависит от множества факторов, какое устройство, на сколько часто итп. в некоторых ноутбуках вентилятор работает на постоянной основе просто на разных оборотах. всё зависит от того на сколько горячий чипсет на материнской плате. современные ноутбуки многие вообще без вентиляторов. на сколько я понимаю у вас на самом деле никакой проблемы нет, а максимум у вас забилась система охлаждения из-за этого затруднён отвод тепла и её нужно просто почистить и возможно заменить термопасту и термопрокладки.\nнадеюсь ответ был полезен. буду благодарен за оценку ответа и если остались вопросы пишите в комментариях с удовольствием отвечу. подписывайтесь на наши ответы и узнаете много полезного.']}

In [12]:
n = 37
print('Before')
print(diasum[n]['dialogue'])
print('After')
pprint(diasum_texts[n])

Before
Пегги: Вы можете порекомендовать зубного врача? У меня болит зуб:/
Шейн: Доктор Смит определенно, в клинике Центра.
Meg: Мне жаль о боли в зубах, Пегги:(
Meg: Я слышал, что он хороший, он поможет тебе.
Peggy: Спасибо, ребята! :*
After
{'texts': ['history: \n'
           'speaker1: вы можете порекомендовать зубного врача? у меня болит '
           'зуб:/\tspeaker2: доктор смит определенно, в клинике центра.',
           'history: speaker1: вы можете порекомендовать зубного врача? у меня '
           'болит зуб:/\t\n'
           'speaker2: доктор смит определенно, в клинике центра.\tspeaker3: '
           'мне жаль о боли в зубах, пегги:(',
           'history: speaker1: вы можете порекомендовать зубного врача? у меня '
           'болит зуб:/\tspeaker2: доктор смит определенно, в клинике '
           'центра.\t\n'
           'speaker3: мне жаль о боли в зубах, пегги:(\tspeaker3: я слышал, '
           'что он хороший, он поможет тебе.',
           'history: speaker1: вы можете по

In [13]:
texts = concatenate_datasets([diasum_texts, yandex_texts])

In [14]:
def explode_texts(elem):
    return {'text': sum(elem['texts'], [])}

texts = texts.map(explode_texts, batched=True, remove_columns=texts.column_names, num_proc=4)

In [15]:
from transformers import AutoTokenizer
checkpoint = "AlexWortega/instruct_rugptMedium"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': [f'speaker{i}: ' for i in range(1, n)] + ['history: ']}
tokenizer.add_special_tokens(special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


10

##### Don't forget to add special tokens to the model 
aka resize embeddings: model.resize_token_embeddings(len(tokenizer)) 

In [16]:
n = np.random.randint(0, len(texts))
inputs = tokenizer(texts[n]['text'])
# tokenizer.convert_ids_to_tokens(inputs["input_ids"])
pprint(tokenizer.decode(inputs['input_ids'][:10]))
pprint(tokenizer.convert_ids_to_tokens(inputs['input_ids'][:10]))

'history: \nspeaker1: сколько стоят услуги контор по типу'
['history: ',
 'Ċ',
 'speaker1: ',
 'ÑģÐºÐ¾Ð»ÑĮÐºÐ¾',
 'ĠÑģÑĤÐ¾ÑıÑĤ',
 'ĠÑĥÑģÐ»ÑĥÐ³Ð¸',
 'ĠÐºÐ¾Ð½',
 'ÑĤÐ¾ÑĢ',
 'ĠÐ¿Ð¾',
 'ĠÑĤÐ¸Ð¿Ñĥ']


In [17]:
def encode_text(elem):
    inputs = tokenizer(elem['text'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    return inputs

encoded_texts = texts.map(encode_text, batched=True, remove_columns=texts.column_names)

In [33]:
# push to HF 
from config import params
name = params['dataset_name']
encoded_texts.push_to_hub(name)

In [ ]:
# type(encoded_texts[0]['ínput_ids'])
# encoded_texts[0]['input_ids'].shape
tokenizer('history: speaker1: hello\t speaker2: hi\t speaker1: how are you?', padding='', return_tensors='pt')

{'input_ids': tensor([[50273, 50264,   398,  5784,    83,   202,   225, 50265, 23961,   202,
           225, 50264,    76,  1170,  2213,  9497,    35]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}